# 분류
> 앙상블 방법(Ensemble Method)
- 정형 데이터의 예측 분석 영역에서 매우 높은 예측 성능으로 인해 애용
- 서로 다른 또는 같은 알고리즘을 단순히 결합한 형태도 있으나,
- 일반적으로 배깅과 부스팅 방식으로 나뉨

> 배깅(Bagging)
- 대표적인 방법 : Random Forest(랜덤 포레스트)

> 부스팅(Boosting)
- 그래디언트 부스팅 (Gradient Boosting) : 뛰어난 예측 성능을 가지고 있지만, 수행 시간이 너무 오래 걸리는 단점 -> 최적화 모델 튜닝 어려움
- XgBoost(eXtra Gradient Boost), LightGBM 등 기존 그래디언트 부스팅을 발전시키면서 수행시간을 단축시킨 알고리즘 계속 등장

# 앙상블 방법(Ensemble Method)
- 매우 많은 여러 개의 약한 학습기(즉, 예측 성능이 상대적으로 떨어지는 학습 알고리즘)를 결합해 확률적 보완과 오류가 발생한 부분에 대한 가중치를 계속 업데이트하면서 예측 성능 향상

## 결정 트리
- 매우 쉽고 유연하게 적용될 수 있는 알고리즘
- 데이터의 스케일링이나 정규화 등의 사전 가공의 영향이 매우 적음
- 하지만 예측 성능을 향상시키기 위해 복잡한 규칙 구조를 가져야 하며, 이로 인한 과적합(overfitting)이 발생해 예측 성능이 저하될 수도 있다는 단점
- 하지만 이러한 단점이 앙상블 기법에서는 오히려 장점으로 작용

- 트리의 깊이(depth)가 깊어질수록 결정트리의 예측 성능이 저하될 가능성 높음(과적합)
- 가능한 한 적은 결정 노드로 높은 예측 정확도를 가지려면 데이터를 분류할 때 최대한 많은 데이터 세트가 해당 분류에 속할 수 있도록 결정 노드의 규칙이 정해져야함

- 복잡한 학습 모델은 결국에는 실제 상황에(테스트 데이터 세트) 유연하게 대처할 수 없어 예측 성능이 떨어질 수밖에 없음
- 트리의 크기를 사전에 제한하는 것이 오히려 성능 튜닝에 더 도움 됌.